已知有以下树状请求，多线程应该如何完成这样的请求？

        root
    A          B
C       D   E

和正常请求的区别在于，请求之间有先后关系，新建一个请求树，然后使用递归处理请求


In [1]:
#include <iostream>
#include <thread>
#include <string>
using namespace std;

In [2]:
class MyThread : public std::thread {
public:
    template <typename Function, typename... Args>
    MyThread(Function&& func, Args&&... args): std::thread(std::forward<Function>(func), std::forward<Args>(args)...) 
    {
        std::cout << "Thread " << this->get_id() << " is starting ";
        // 以下代码判断参数是否为空，不为空输出其data成员
        // if (((args != nullptr),...)) {
        //     ((std::cout << args->data << ' '), ...);
        // }
        std::cout << std::endl;
    }
    ~MyThread() {
        std::cout << "Thread " << std::this_thread::get_id() << " is exiting.\n";
        // if (joinable()) {
        //     _STD terminate();
        // }
    }
};

In [3]:
struct Node {
    std::string data;
    Node* left;
    Node* right;
};

Node* root = new Node();
root->data = "root";
root->left = new Node();
root->left->data = "A";
root->right = new Node();
root->right->data = "B";

root->left->left = new Node();
root->left->left->data="C";
root->left->right = new Node();
root->left->right->data = "D";

root->right->left = new Node();
root->right->left->data = "E";

In [4]:
// ---------------------遍历1:join跟随某个分支，则线程启动先在某个分支启动------------------
void PreOrderTraversal(Node* node) {
    if (node == nullptr) {
        return;
    }

    // Print the data of node
    std::cout << "Enter:" << node->data << std::endl;

    if (node->left != nullptr)
    {
        //PreOrderTraversal(node->left);                           // 原生调用
        //std::thread left_thread(PreOrderTraversal, node->left);  // std::thread
        MyThread left_thread(PreOrderTraversal, node->left);       // 自定义多线程类

        left_thread.join();
    }
    if (node->right != nullptr)
    {
        //PreOrderTraversal(node->right);
        //std::thread right_thread(PreOrderTraversal, node->right);
        MyThread right_thread(PreOrderTraversal, node->right);

        right_thread.join();
    }
    //------------------------------------------------------------------------------------------

    // ----------------------遍历2: join在最后，线程启动在各个分支以不同速度推进----------------
    //MyThread left_thread(PreOrderTraversal, node->left);       // 自定义多线程类
    //MyThread right_thread(PreOrderTraversal, node->right);
    //left_thread.join();
    //right_thread.join();
    //------------------------------------------------------------------------------------------

    std::cout <<"Exit:" << node->data << std::endl;
}

In [5]:
PreOrderTraversal(root);

Enter:root
Thread 140284109932288 is starting 
Enter:A
Thread 140283899541248 is starting 
Enter:C
Exit:C
Thread 140284109932288 is exiting.
Thread 140283899541248 is starting 
Enter:D
Exit:D
Thread 140284109932288 is exiting.
Exit:A
Thread 140284366404096 is exiting.
Thread 140284109932288 is starting 
Enter:B
Thread 140283899541248 is starting 
Enter:E
Exit:E
Thread 140284109932288 is exiting.
Exit:B
Thread 140284366404096 is exiting.
Exit:root


In [6]:
// ----------------------遍历2: join在最后，线程启动在各个分支以不同速度推进----------------
void PreOrderTraversal2(Node* node) {
    if (node == nullptr) {
        return;
    }

    // Print the data of node
    std::cout << "Enter:" << node->data << std::endl;

    MyThread left_thread(PreOrderTraversal2, node->left);       // 自定义多线程类
    MyThread right_thread(PreOrderTraversal2, node->right);
    left_thread.join();
    right_thread.join();

    std::cout <<"Exit:" << node->data << std::endl;
}

In [7]:
PreOrderTraversal2(root);

Enter:root
Thread 140284109932288 is starting 
Thread 140283899541248 is starting 
Enter:Enter:B
A
Thread 140283891148544 is starting 
Thread 140283882755840 is starting 
Enter:Thread C
140283874363136 is starting 
Thread 140283865970432 is starting 
Enter:E
Enter:Thread D
Thread 140283857577728 is starting 
140283849185024 is starting 
Thread 140283631105792 is starting 
Thread 140283622713088 is starting 
Exit:Thread C
Thread 140283882755840 is exiting.
Thread 140283882755840 is exiting.
140283614320384 is starting 
Thread 140283605927680 is starting 
Exit:E
Thread 140283891148544 is exiting.
Thread 140283891148544 is exiting.
Exit:D
Thread 140283865970432 is exiting.
Thread 140283865970432 is exiting.
Exit:B
Thread 140283899541248 is exiting.
Thread 140283899541248 is exiting.
Exit:A
Thread 140284109932288 is exiting.
Thread 140284109932288 is exiting.
Exit:root
Thread 140284366404096 is exiting.
Thread 140284366404096 is exiting.
